1. The partial derivatives are as follows:
\begin{align}
\frac{\partial f}{\partial x} &= \cos(x+2y),\\
\frac{\partial f}{\partial y} &= 2\cos(x+2y),\\
\frac{\partial^2 f}{\partial x^2} &= -\sin(x+2y),\\
\frac{\partial^2 f}{\partial y^2} &= -4\sin(x+2y),\\
\frac{\partial^2 f}{\partial x\partial y} &= -2\sin(x+2y)
\end{align}
So the second order Taylor series about $(\pi/2, 0)$ is
\begin{align*}
f(x, y)&\approx \sin(\pi/2) + (x-\pi/2)\cos(\pi/2) + 2y\cos(\pi/2) - \frac{(x-\pi/2)^2}{2}\sin(\pi/2) - 4\frac{y^2}{2}\sin(\pi/2) - 4\frac{(x-\pi/2)y}{2}\sin(\pi/2)\\
&= 1 - \frac{(x+2y-\pi/2)^2}{2}
\end{align*}
Therefore $f(1.5, 0.2)\approx 1 - \frac{(1.9-\pi/2)^2}{2} \approx \boxed{0.946}$.

2. (a) Explicit Euler Method

In [17]:
# set up variables & functions
h = 1
def y_prime(t, y):
    return 1 + t + y/2

t = 0
y = 1
while t < 2:
    t += h
    y += h*y_prime(t, y)

print("Explicit Euler Method:")
print("y(2) =", y)

Explicit Euler Method:
y(2) = 8.25


2. (b) Implicit Euler Method
We first need to implicitly solve the equation
\begin{align*}
y(t) &= y(t-h) + hy'(t)\\
&= y(t-h) + h + ht + hy(t)/2\\
&\implies\\
y(t)\left(1-\frac{h}2\right)&= y(t-h) + h + ht\\
&\Longleftrightarrow\\
y(t) &= \frac{2y(t-h) + 2h + 2ht}{2-h}
\end{align*}

In [18]:
# set up variables
h = 1

# Just iterate up
y = 1
t = 0
while t < 2:
    t += h
    y = (2*y + 2*h + 2*h*t)/(2-h)
print("Implicit Euler Method:")
print("y(2) =", y)

Implicit Euler Method:
y(2) = 18.0


3. (a) Setting $v=u'$ we have $v'=u''$ so the equation becomes
$$v' = 3t-\gamma v - 2\cos(u)$$
So $g(u,v) = 3t-\gamma v - 2\cos(u)$. Also, $v(0) = u'(0) = -1$.

   (b) Code follows:

In [37]:
import numpy as np

gamma = 1 # We aren't given this value, so just assuming it is 1.

def g(t, u, v):
    return 3*t - gamma * v - 2*np.cos(u)

def f(t, y):
    # y[0] = u
    # y[1] = v
    return np.array([y[1], g(t, *y)])

def Huens(y, t, h, f):
    return [y + 1/2 * h * (f(t, y) + f(t, y + h*f(t, y))), t + h]

y = np.array([1, -1]) # initial conditions: u=1, v=-1
h = 1/5
t = 0
while t < 5:
    y, t = Huens(y, t, h, f)
print("For h = 1/5:")
print(f"u = {y[0]}, v = {y[1]}")

y = np.array([1, -1]) # initial conditions: u=1, v=-1
h = 1/20
t = 0
while t < 5:
    y, t = Huens(y, t, h, f)
print("For h = 1/20:")
print(f"u = {y[0]}, v = {y[1]}")

For h = 1/5:
u = 20.572682932821422, v = 11.532886974365958
For h = 1/20:
u = 22.36896916546019, v = 12.099340593424884


I have just assumed $\gamma = 1$ because it wasn't given in the problem. For $h=1/20$ you gain much more accuracy. If you take $h=1/1000$ you get $u\approx 22.186$ and $v\approx 12.000$. The values for $h=1/5$ are about $4-7\%$ off of this value while the values for $h=1/20$ are less than $1\%$ off.

4. (a) The fixed point iteration system is:
\begin{align*}
x_{n+1} &= \sqrt{1-y_n^2} \\
y_{n+1} &= \frac15 \sqrt{9-5x_n^2}
\end{align*}
Which is equivalent to
\begin{align*}
x_{n+1}^2 + y_n^2 &= 1\\
5x_n^2 + 25y_{n+1}^2 &= 9
\end{align*}
Which is the original system (assuming $x_n, y_n$ are a Cauchy sequence and $n$ tends towards infinity (so $x_{n+1} = x_n, y_{n+1} = y_n$)).

   (b) We have
$$\textbf{g} = \binom{\sqrt{1-y^2}}{\frac15\sqrt{9-5x^2}}$$
So the Jacobian matrix $J(\textbf{g})$ is
$$\begin{bmatrix}
0 & -\frac{y}{\sqrt{1-y^2}} \\ 
-\frac{x}{\sqrt{9-5x^2}} & 0
\end{bmatrix}$$

   (c) The $1$-norm and $\infty$-norm are both equal to $\max\left(\frac{y}{\sqrt{1-y^2}}, \frac{x}{\sqrt{9-5x^2}}\right)$. This is less than $1$ iff
\begin{align*}
y < \sqrt{1-y^2},\quad &\text{and}\quad x < \sqrt{9-5x^2}\\
&\Longleftrightarrow\\
y^2 < \frac12,\quad &\text{and}\quad x^2 < \frac{3}{2}
\end{align*}
Which is true inside the region $R$.

   (d) This means that $\textbf{g}$ is Lipschitz continuous (the exercise proves it's a contraction map in $R$, and we already knew it was continuos). Therefore by the Banach fixed-point theorem there is exactly one fixed point in $R$ and our sequence $x_n, y_n$ defined above is a Cauchy sequence (successive $x_n, y_n$ get arbitrarily close and approach some limit point—the fixed point).
   
   (e) Code follows:

In [54]:
import math

x, y = 0.5, 0.3 # initial values

def g(x, y):
    return math.sqrt(1-y**2), 1/5 * math.sqrt(9 - 5*x**2)

epsilon = 1e-8
prev_x, prev_y = x, y
x, y = g(x, y)
iters = 0
while (prev_x - x)**2 + (prev_y - y)**2 > epsilon**2:
    iters+=1
    prev_x, prev_y = x, y
    x, y = g(x, y)
print(f"Took {iters} iterations.")
print(f"(x, y) = {(x, y)}")
print("x^2 + y^2 =", x**2 + y**2)
print("5x^2 + 25y^2 =", 5*x**2 + 25*y**2)

Took 22 iterations.
(x, y) = (0.8944271922592694, 0.4472135980186649)
x^2 + y^2 = 1.0000000045056
5x^2 + 25y^2 = 9.000000067583999


5. (a) We have the system
$$f(\textbf{x}) = 0$$
where
$$f(x) = \begin{pmatrix}
x_1^2+x_2^2+x_3^2 - 1\\ 
2x_1^2+x_2^2 - 4x_3\\ 
3x_1^2-4x_2+x_3^2
\end{pmatrix}$$
so
$$J(\textbf{x}) = \begin{bmatrix}
2x_1 & 2x_2 & 2x_3\\ 
4x_1 & 2x_2 & -4\\ 
6x_1 & -4 & 2x_3
\end{bmatrix}$$
Then
$$J_0 = \begin{bmatrix}
1 & 1 & 1\\ 
2 & 1 & -4\\ 
3 & -4 & 1
\end{bmatrix}$$
   
   (b) Code follows:

In [68]:
import numpy as np

x = np.array([1/2, 1/2, 1/2]) # initial guess
def f(x):
    return np.array([x[0]**2 + x[1]**2 + x[2]**2 - 1,
                    2*x[0]**2 + x[1]**2 - 4*x[2],
                    3*x[0]**2 - 4*x[1] + x[2]**3])
def J(x):
    return np.array([[2*x[0], 2*x[1], 2*x[2]],
                     [4*x[0], 2*x[1], -4],
                     [6*x[0], -4, 2*x[2]]])
def Newton(x):
    J_inverse = np.linalg.inv(J(x))
    return x - np.matmul(J_inverse, f(x))

tolerance = 1e-7
prev_x = np.copy(x)
x = Newton(x)
iterations = 0
while sum((x - prev_x)**2) > tolerance**2:
    prev_x = np.copy(x)
    x = Newton(x)
    iterations += 1
print(f"(x, y, z) = {x[0], x[1], x[2]}")
print(f"It took {iterations} iterations.")

(x, y, z) = (0.7921987726309627, 0.4835800410479299, 0.372251861703981)
It took 4 iterations.


    (c) What if we don't update our Jacobian matrix? Here's the code for that:

In [72]:
x = np.array([1/2, 1/2, 1/2]) # initial guess
def f(x):
    return np.array([x[0]**2 + x[1]**2 + x[2]**2 - 1,
                    2*x[0]**2 + x[1]**2 - 4*x[2],
                    3*x[0]**2 - 4*x[1] + x[2]**3])
def J(x):
    return np.array([[2*x[0], 2*x[1], 2*x[2]],
                     [4*x[0], 2*x[1], -4],
                     [6*x[0], -4, 2*x[2]]])
J0 = J(x)
J_inverse = np.linalg.inv(J0)
def Newton(x):
    return x - np.matmul(J_inverse, f(x))

tolerance = 1e-7
prev_x = np.copy(x)
x = Newton(x)
iterations = 0
while sum((x - prev_x)**2) > tolerance**2:
    prev_x = np.copy(x)
    x = Newton(x)
    iterations += 1
print(f"(x, y, z) = {x[0], x[1], x[2]}")
print(f"It took {iterations} iterations.")

(x, y, z) = (0.7921988030681704, 0.48358004128817467, 0.37225186166164975)
It took 28 iterations.


It does still converge, but it takes 7x more iterations of the algorithm to converge. But let's time it, because the algorithm is quicker to run even if it takes more iterations:

In [81]:
import time
x = np.array([1/2, 1/2, 1/2])

def f(x):
    return np.array([x[0]**2 + x[1]**2 + x[2]**2 - 1,
                    2*x[0]**2 + x[1]**2 - 4*x[2],
                    3*x[0]**2 - 4*x[1] + x[2]**3])
def J(x):
    return np.array([[2*x[0], 2*x[1], 2*x[2]],
                     [4*x[0], 2*x[1], -4],
                     [6*x[0], -4, 2*x[2]]])
def Newton_partb(x):
    J_inverse = np.linalg.inv(J(x))
    return x - np.matmul(J_inverse, f(x))

J0 = J(x)
J0_inverse = np.linalg.inv(J0)
def Newton_partc(x):
    return x - np.matmul(J0_inverse, f(x))

trials = 100000
t = time.time()
for i in range(trials):
    Newton_partb(x)
time_partb = time.time() - t
print(f"It took {time_partb/trials} sec per iteration for the first algorithm.")
t = time.time()
for i in range(trials):
    Newton_partc(x)
time_partc = time.time() - t
print(f"It took {time_partc/trials} sec per iteration for the second algorithm.")

print(f"\nThis means the second algorithm runs {time_partb/time_partc} times faster.")

It took 6.936999320983887e-05 sec per iteration for the first algorithm.
It took 7.710011005401611e-06 sec per iteration for the second algorithm.

This means the second algorithm runs 8.997392242532268 times faster.


My method to time the algorithms isn't perfect, but the second algorithm was much faster per iteration than the second. Even though it took 7x the number of iterations, method $(c)$ actually runs just as fast as method $(b)$ because of this.